# Schema Linking Embeddings: Running Experiments

## Download Necessary Code

In [ ]:
!pip -q install fasttext
!git clone https://github.com/geokats/schema-linking-embeddings.git
!pip -q install -r schema-linking-embeddings/requirements.txt
!tar xvjf /content/schema-linking-embeddings/wikisql/data.tar.bz2 -C /content/schema-linking-embeddings/wikisql/
!wget https://raw.githubusercontent.com/facebookresearch/fastText/master/python/doc/examples/bin_to_vec.py

## Import Libraries

In [ ]:
import os
import nltk
import json
import argparse
import random
import sys
import pandas as pd
from gensim.models import KeyedVectors
from tqdm.auto import tqdm
import fasttext
import fasttext.util

import warnings
warnings.filterwarnings('ignore')

os.chdir("/content/schema-linking-embeddings/")
from alignment.align import align_embeddings
from util import wikisql_table_to_df, create_table_emb, vector_align, create_gt
from util import get_row_matches, get_col_matches, add_aligned_vectors, get_stats, get_rec, get_prec, get_f_score
os.chdir("/content/")

In [ ]:
nltk.download('punkt')

## Prepare Schema Linking Data

In [ ]:
os.chdir("/content/schema-linking-embeddings/")

!python create_data.py \
  -i /content/schema-linking-embeddings/wikisql/data/train.jsonl \
  -t /content/schema-linking-embeddings/wikisql/data/train.tables.jsonl \
  -o /content/schema-linking-embeddings/wikisql/data/train.sl.jsonl 

!python create_data.py \
  -i /content/schema-linking-embeddings/wikisql/data/dev.jsonl \
  -t /content/schema-linking-embeddings/wikisql/data/dev.tables.jsonl \
  -o /content/schema-linking-embeddings/wikisql/data/dev.sl.jsonl 

os.chdir("/content/")

## Define Experiment's Parameters

In [ ]:
EMBEDDING_ALG = "fasttext"
EMBEDDING_DIM = 100
COL_THRESHOLD = 0.5
ROW_THRESHOLD = 0.6
LOAD_QUERIES = 1000

TEMP_DIR = "/content/tmp"
if not os.path.isdir(TEMP_DIR):
  !mkdir {TEMP_DIR}

#Initialize statistics
col_stats = {
    'tab_tp' : 0, 'tab_fp' : 0, 'tab_fn' : 0,
    'pre_tp' : 0, 'pre_fp' : 0, 'pre_fn' : 0,
}

row_stats = {
    'tab_tp' : 0, 'tab_fp' : 0, 'tab_fn' : 0,
    'pre_tp' : 0, 'pre_fp' : 0, 'pre_fn' : 0,
}

## Download Pre-trained Embeddings

In [ ]:
if EMBEDDING_ALG == 'fasttext':
  if not os.path.isfile("/content/cc.en.300.bin"):
    #Download file if we haven't already
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
    !gzip -d /content/cc.en.300.bin.gz
  
  if not os.path.isfile(f"/content/cc.en.{EMBEDDING_DIM}.bin"):
    #Resize file if necessary
    ft = fasttext.load_model('/content/cc.en.300.bin')
    fasttext.util.reduce_model(ft, EMBEDDING_DIM)
    pre_trained_emb_file = f"/content/cc.en.{EMBEDDING_DIM}.bin"
    ft.save_model(pre_trained_emb_file)
  
  pre_trained_emb_file = f"/content/cc.en.{EMBEDDING_DIM}.bin"

  if pre_trained_emb_file.endswith(".bin"):
    #Convert .bin file to .vec
    new_name = pre_trained_emb_file.replace(".bin", ".vec")
    !python bin_to_vec.py {pre_trained_emb_file} > {new_name}
    pre_trained_emb_file = new_name

elif EMBEDDING_ALG == 'word2vec':
  if EMBEDDING_DIM == 100:
    if not os.path.isfile("/content/enwiki_20180420_100d.txt.bz2"):
      #Download file if we haven't already
      !wget http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_100d.txt.bz2
      !tar -xvf /content/enwiki_20180420_100d.txt.bz2
      pre_trained_emb_file = f"/content/enwiki_20180420_100d.txt"
  
  elif EMBEDDING_DIM == 300:
    if not os.path.isfile("/content/enwiki_20180420_300d.txt.bz2"):
      #Download file if we haven't already
      !wget http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_300d.txt.bz2
      !tar -xvf /content/enwiki_20180420_300d.txt.bz2
      pre_trained_emb_file = f"/content/enwiki_20180420_300d.txt"

## Run Experiment

In [ ]:
#Load data
unique_words = set()

queries_per_table = {}
with open("/content/schema-linking-embeddings/wikisql/data/train.sl.jsonl", 'r') as f:
    for i, line in enumerate(f):
        obj = json.loads(line)
        tid = obj['table_id']
        if tid in queries_per_table:
          queries_per_table[tid].append(obj)
        else:
          queries_per_table[tid] = [obj]

        #Add all words to unique words
        words = obj['question']
        for word in words:
              unique_words.add(word)

        if i + 1 == LOAD_QUERIES:
          break

tables = {}
with open("/content/schema-linking-embeddings/wikisql/data/train.tables.jsonl", 'r') as f:
    for line in f:
        obj = json.loads(line)
        if obj['id'] not in queries_per_table:
          continue

        tables[obj['id']] = obj
        
        #Add all words to unique words
        for h in obj['header']:
          words = nltk.word_tokenize(h)
          for word in words:
            unique_words.add(word)
        for row in obj['rows']:
          for c in row:
            words = nltk.word_tokenize(str(c))
            for word in words:
              unique_words.add(word)

In [ ]:
new_emb_file = pre_trained_emb_file + ".lite"

lines = []

with open(pre_trained_emb_file, 'r') as inpf:
  for i, line in enumerate(inpf):
    if i == 0:
      continue
    else:
      l = line.split(' ')
      if l[0] in unique_words:
        lines.append(line)

with open(new_emb_file, 'w') as outf:
  outf.write(f"{len(lines)} {EMBEDDING_DIM}\n")
  for line in lines:
    outf.write(line)
    
pre_trained_emb_file = new_emb_file
    

In [ ]:
#Load pre-trained embeddings
kv_pre = KeyedVectors.load_word2vec_format(pre_trained_emb_file)
vec_pre = kv_pre.vectors
words_pre_set = set(kv_pre.key_to_index.keys())
idx_pre = kv_pre.key_to_index

In [ ]:
for table_id, queries in tqdm(queries_per_table.items()):
  table = tables[table_id]
  tdf = wikisql_table_to_df(table)
  
  #Create local table embeddings
  create_table_emb(tdf, os.path.join(TEMP_DIR, "tmp.emb"), EMBEDDING_ALG, EMBEDDING_DIM, TEMP_DIR)
  kv_tab = KeyedVectors.load_word2vec_format(os.path.join(TEMP_DIR, "tmp.emb"))
  vec_tab = kv_tab.vectors
  words_tab_set = set(kv_tab.key_to_index.keys())
  idx_tab = kv_tab.key_to_index

  #Find anchor words
  anchors = words_tab_set & words_pre_set #Common words are the intersection
  pairs = [(idx_pre[w], idx_tab[w]) for w in anchors]
  
  #Find alignment matrix of pre-trained vectors to the table embedding space
  R = align_embeddings(vec_pre, vec_tab, pairs)

  for query in queries:
    #Construct ground truths
    gt_col_matches, gt_row_matches = create_gt(query['sql'], query['schema_links'], tdf)
    #Tokenize NLQ
    nlq = query['question']
    tokens = nltk.word_tokenize(nlq)
    # #Align NLQ pre-trained vectors to local table embeddings
    kv = add_aligned_vectors(kv_tab, tokens, vec_pre, idx_pre, R)

    #Predictions for columns by localy aligned embeddings
    col_matches = get_col_matches(tokens, tdf, kv_tab, threshold=COL_THRESHOLD)
    tp, fp, fn = get_stats(col_matches, gt_col_matches)
    col_stats['tab_tp'] += tp
    col_stats['tab_fp'] += fp
    col_stats['tab_fn'] += fn
    #Predictions for rows by localy aligned embeddings
    row_matches = get_row_matches(tokens, tdf, kv_tab, threshold=COL_THRESHOLD)
    tp, fp, fn = get_stats(row_matches, gt_row_matches)
    row_stats['tab_tp'] += tp
    row_stats['tab_fp'] += fp
    row_stats['tab_fn'] += fn

    #Predictions for columns by pre-trained embeddings
    col_matches = get_col_matches(tokens, tdf, kv_pre, threshold=ROW_THRESHOLD)
    tp, fp, fn = get_stats(col_matches, gt_col_matches)
    col_stats['pre_tp'] += tp
    col_stats['pre_tp'] += fp
    col_stats['pre_fn'] += fn
    #Predictions for rows by pre-trained embeddings
    row_matches = get_row_matches(tokens, tdf, kv_pre, threshold=ROW_THRESHOLD)
    tp, fp, fn = get_stats(row_matches, gt_row_matches)
    row_stats['pre_tp'] += tp
    row_stats['pre_fp'] += fp
    row_stats['pre_fn'] += fn       

In [ ]:
tab_col_prec, pre_col_prec = get_prec(col_stats)
tab_col_rec, pre_col_rec = get_rec(col_stats)
tab_col_f = get_f_score(tab_col_prec, tab_col_rec)
pre_col_f = get_f_score(pre_col_prec, pre_col_rec)

tab_row_prec, pre_row_prec = get_prec(row_stats)
tab_row_rec, pre_row_rec = get_rec(row_stats)
tab_row_f = get_f_score(tab_row_prec, tab_row_rec)
pre_row_f = get_f_score(pre_row_prec, pre_row_rec)

print(f"Locally Aligned col prec: {tab_col_prec}")
print(f"Locally Aligned col rec: {tab_col_rec}")
print(f"Locally Aligned col f1: {tab_col_f}\n")

print(f"Locally Aligned row prec: {tab_row_prec}")
print(f"Locally Aligned row rec: {tab_row_rec}")
print(f"Locally Aligned row f1: {tab_row_f}\n")

print(f"Pre-trained col prec: {pre_col_prec}")
print(f"Pre-trained col rec: {pre_col_rec}")
print(f"Pre-trained col f1: {pre_col_f}\n")

print(f"Pre-trained row prec: {pre_row_prec}")
print(f"Pre-trained row rec: {pre_row_rec}")
print(f"Pre-trained row f1: {pre_row_f}\n")